In [47]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///stock.db', echo = True)
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
from sqlalchemy import Column, Integer, String, Float, Date, DateTime, SmallInteger, Boolean
from sqlalchemy import ForeignKey
from sqlalchemy.sql import select, text
from sqlalchemy.orm import relationship
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [35]:
class StockData(Base):
    """銘柄の情報を持つクラス"""
    __tablename__ = 'stock'
    code = Column(Integer, primary_key=True)  # 銘柄コード
    name = Column(String(64), nullable=False)  # 銘柄の名前
    category_id = Column(Integer, nullable=True, index=True)  # 業種
    activated = Column(Boolean, default=False)  # backtest対象か否か
    created_at = Column(DateTime, default=datetime.now())  # 作成日時
    updated_at = Column(DateTime, default=datetime.now(), onupdate=datetime.now())  # 更新日時

class History(Base):
    """外部キーの記載方法
    http://docs.sqlalchemy.org/en/latest/orm/relationship_persistence.html
    ここに__table_args__とForeignKeyConstraintを使う例がある
    """
    __tablename__ = 'history'
    code = Column(Integer,ForeignKey('stock.code'), primary_key=True, )  # 銘柄コード
    date = Column(Date, primary_key=True, unique=False)  # 日付
    #raw_close_price = Column(Float, nullable=False)  # 終値（株式分割調整前）
    Open = Column(Float, nullable=False)  # 始値
    High = Column(Float, nullable=False)  # 高値
    Low = Column(Float, nullable=False)  # 安値
    Close = Column(Float, nullable=False)  # 終値
    Volume = Column(Integer, nullable=False)  # 出来高
    
    stock = relationship('StockData', back_populates='history')
    
    
StockData.code = relationship("History", order_by=History.code, back_populates="StockData")
Base.metadata.create_all(engine)

2018-01-21 15:52:53,834 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-01-21 15:52:53,835 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 15:52:53,837 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-01-21 15:52:53,838 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 15:52:53,841 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stock")
2018-01-21 15:52:53,842 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 15:52:53,845 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("history")
2018-01-21 15:52:53,847 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 15:52:53,850 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE stock (
	code INTEGER NOT NULL, 
	name VARCHAR(64) NOT NULL, 
	category_id INTEGER, 
	activated BOOLEAN, 
	created_at DATETIME, 
	updated_at DATETIME, 
	PRIMARY KEY (code), 
	CHECK (activated IN (0, 1))
)


2018-01-21 15:52:53,853 INFO sqlalchemy.engine.base.Engine (

In [36]:
files = os.listdir('dd')
for file in files:
    if(os.path.isfile('dd/' + file)):
        hist = pd.read_csv('dd/' + file,
                           names=["date", "Open", "High", "Low", "Close", "Volume"],
                           usecols=range(6))
        stk = pd.read_csv('dd/' + 'info' + '/' + file, names=["code", "name", "category_id"])
        codes = np.repeat(np.array(stk["code"]), (len(hist["date"])))
        hist['code'] = codes
        lst = hist.columns.tolist()
        lst.remove('code')
        lst.insert(0, 'code')
        hist = hist[lst]
        stk.to_sql('stock', engine, index=False, if_exists='append')
        hist.to_sql('history', engine, index=False, if_exists='append')

2018-01-21 15:52:54,345 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("stock")
2018-01-21 15:52:54,348 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 15:52:54,353 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-21 15:52:54,355 INFO sqlalchemy.engine.base.Engine INSERT INTO stock (code, name, category_id) VALUES (?, ?, ?)
2018-01-21 15:52:54,357 INFO sqlalchemy.engine.base.Engine (1301, '(株)極洋', '水産・農林業')
2018-01-21 15:52:54,365 INFO sqlalchemy.engine.base.Engine COMMIT
2018-01-21 15:52:54,560 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("history")
2018-01-21 15:52:54,561 INFO sqlalchemy.engine.base.Engine ()
2018-01-21 15:52:54,568 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-21 15:52:54,605 INFO sqlalchemy.engine.base.Engine INSERT INTO history (code, date, "Open", "High", "Low", "Close", "Volume") VALUES (?, ?, ?, ?, ?, ?, ?)
2018-01-21 15:52:54,606 INFO sqlalchemy.engine.base.Engine ((1301, '2000/01/04', 1350.0, 1350.0, 1330.0, 1330.0, 163

## 生SQLテスト ( Queryを使うとmapper初期化エラー）

In [39]:
rows = engine.execute('SELECT * FROM history')

# 表示
for row in rows:
    print(row)

2018-01-21 21:13:47,636 INFO sqlalchemy.engine.base.Engine SELECT * FROM history
2018-01-21 21:13:48,574 INFO sqlalchemy.engine.base.Engine ()
(1301, '2000/01/04', 1350.0, 1350.0, 1330.0, 1330.0, 163000)
(1301, '2000/01/05', 1370.0, 1450.0, 1350.0, 1350.0, 218000)
(1301, '2000/01/06', 1360.0, 1380.0, 1350.0, 1350.0, 194000)
(1301, '2000/01/07', 1380.0, 1410.0, 1360.0, 1370.0, 211000)
(1301, '2000/01/11', 1370.0, 1380.0, 1350.0, 1350.0, 236000)
(1301, '2000/01/12', 1360.0, 1380.0, 1350.0, 1370.0, 251000)
(1301, '2000/01/13', 1380.0, 1430.0, 1370.0, 1390.0, 183000)
(1301, '2000/01/14', 1400.0, 1450.0, 1380.0, 1410.0, 493000)
(1301, '2000/01/17', 1450.0, 1630.0, 1440.0, 1600.0, 391000)
(1301, '2000/01/18', 1570.0, 1570.0, 1470.0, 1520.0, 195000)
(1301, '2000/01/19', 1470.0, 1570.0, 1470.0, 1520.0, 173000)
(1301, '2000/01/20', 1570.0, 1640.0, 1540.0, 1630.0, 388000)
(1301, '2000/01/21', 1560.0, 1600.0, 1510.0, 1530.0, 337000)
(1301, '2000/01/24', 1530.0, 1570.0, 1470.0, 1470.0, 152000)
(13

(1301, '2001/06/14', 1500.0, 1520.0, 1470.0, 1520.0, 273000)
(1301, '2001/06/15', 1510.0, 1530.0, 1490.0, 1520.0, 321000)
(1301, '2001/06/18', 1540.0, 1550.0, 1510.0, 1530.0, 220000)
(1301, '2001/06/19', 1540.0, 1600.0, 1510.0, 1530.0, 573000)
(1301, '2001/06/20', 1540.0, 1560.0, 1530.0, 1560.0, 302000)
(1301, '2001/06/21', 1560.0, 1580.0, 1550.0, 1550.0, 293000)
(1301, '2001/06/22', 1560.0, 1560.0, 1530.0, 1540.0, 179000)
(1301, '2001/06/25', 1540.0, 1560.0, 1530.0, 1530.0, 165000)
(1301, '2001/06/26', 1530.0, 1610.0, 1530.0, 1610.0, 324000)
(1301, '2001/06/27', 1570.0, 1680.0, 1570.0, 1650.0, 891000)
(1301, '2001/06/28', 1630.0, 1640.0, 1570.0, 1580.0, 274000)
(1301, '2001/06/29', 1650.0, 1650.0, 1590.0, 1620.0, 277000)
(1301, '2001/07/02', 1590.0, 1600.0, 1520.0, 1580.0, 247000)
(1301, '2001/07/03', 1570.0, 1580.0, 1560.0, 1580.0, 160000)
(1301, '2001/07/04', 1570.0, 1580.0, 1530.0, 1530.0, 244000)
(1301, '2001/07/05', 1510.0, 1560.0, 1490.0, 1560.0, 371000)
(1301, '2001/07/06', 156

(1301, '2003/09/29', 1890.0, 1890.0, 1850.0, 1870.0, 248000)
(1301, '2003/09/30', 1870.0, 1880.0, 1850.0, 1880.0, 302000)
(1301, '2003/10/01', 1860.0, 1890.0, 1840.0, 1850.0, 676000)
(1301, '2003/10/02', 1880.0, 1890.0, 1860.0, 1870.0, 850000)
(1301, '2003/10/03', 1880.0, 2040.0, 1880.0, 2030.0, 3208000)
(1301, '2003/10/06', 2010.0, 2010.0, 1960.0, 1970.0, 1136000)
(1301, '2003/10/07', 1970.0, 1980.0, 1950.0, 1960.0, 683000)
(1301, '2003/10/08', 1960.0, 1960.0, 1940.0, 1940.0, 490000)
(1301, '2003/10/09', 1940.0, 1940.0, 1890.0, 1900.0, 752000)
(1301, '2003/10/10', 1900.0, 1920.0, 1870.0, 1910.0, 509000)
(1301, '2003/10/14', 1930.0, 1940.0, 1880.0, 1890.0, 554000)
(1301, '2003/10/15', 1890.0, 1890.0, 1860.0, 1870.0, 403000)
(1301, '2003/10/16', 1870.0, 1900.0, 1870.0, 1880.0, 277000)
(1301, '2003/10/17', 1900.0, 1940.0, 1890.0, 1910.0, 697000)
(1301, '2003/10/20', 1950.0, 1950.0, 1920.0, 1940.0, 715000)
(1301, '2003/10/21', 1940.0, 1960.0, 1890.0, 1910.0, 637000)
(1301, '2003/10/22', 1

(1301, '2008/08/05', 2000.0, 2070.0, 2000.0, 2010.0, 379000)
(1301, '2008/08/06', 2060.0, 2110.0, 2050.0, 2110.0, 541000)
(1301, '2008/08/07', 2110.0, 2110.0, 2030.0, 2030.0, 210000)
(1301, '2008/08/08', 2010.0, 2050.0, 2010.0, 2050.0, 147000)
(1301, '2008/08/11', 2050.0, 2060.0, 2030.0, 2050.0, 78000)
(1301, '2008/08/12', 2050.0, 2080.0, 2000.0, 2000.0, 253000)
(1301, '2008/08/13', 2000.0, 2030.0, 1950.0, 1980.0, 264000)
(1301, '2008/08/14', 1960.0, 1970.0, 1920.0, 1950.0, 281000)
(1301, '2008/08/15', 1930.0, 1980.0, 1930.0, 1950.0, 120000)
(1301, '2008/08/18', 1950.0, 2020.0, 1940.0, 1980.0, 217000)
(1301, '2008/08/19', 1960.0, 1990.0, 1940.0, 1990.0, 159000)
(1301, '2008/08/20', 2010.0, 2010.0, 1980.0, 2010.0, 255000)
(1301, '2008/08/21', 2000.0, 2000.0, 1970.0, 1980.0, 175000)
(1301, '2008/08/22', 1960.0, 1980.0, 1950.0, 1970.0, 112000)
(1301, '2008/08/25', 1990.0, 2000.0, 1970.0, 1980.0, 150000)
(1301, '2008/08/26', 1940.0, 1950.0, 1930.0, 1940.0, 128000)
(1301, '2008/08/27', 1940

(1301, '2012/02/10', 1910.0, 1930.0, 1900.0, 1910.0, 386000)
(1301, '2012/02/13', 1920.0, 1950.0, 1920.0, 1950.0, 388000)
(1301, '2012/02/14', 1940.0, 1950.0, 1920.0, 1940.0, 340000)
(1301, '2012/02/15', 1940.0, 1970.0, 1930.0, 1940.0, 826000)
(1301, '2012/02/16', 1950.0, 1990.0, 1940.0, 1970.0, 713000)
(1301, '2012/02/17', 1980.0, 1980.0, 1960.0, 1980.0, 529000)
(1301, '2012/02/20', 1990.0, 2010.0, 1980.0, 2010.0, 540000)
(1301, '2012/02/21', 1990.0, 2010.0, 1980.0, 1990.0, 364000)
(1301, '2012/02/22', 1990.0, 2000.0, 1980.0, 1980.0, 222000)
(1301, '2012/02/23', 1990.0, 2010.0, 1990.0, 1990.0, 528000)
(1301, '2012/02/24', 1990.0, 2020.0, 1990.0, 2000.0, 529000)
(1301, '2012/02/27', 2010.0, 2010.0, 1990.0, 2000.0, 209000)
(1301, '2012/02/28', 1990.0, 2000.0, 1990.0, 2000.0, 176000)
(1301, '2012/02/29', 2000.0, 2020.0, 1980.0, 1980.0, 499000)
(1301, '2012/03/01', 1990.0, 2010.0, 1970.0, 1980.0, 354000)
(1301, '2012/03/02', 1990.0, 2030.0, 1980.0, 2020.0, 401000)
(1301, '2012/03/05', 204

...

(1306, '2007/11/16', 1491.0, 1496.0, 1478.0, 1493.0, 1784400)
(1306, '2007/11/19', 1486.0, 1509.0, 1473.0, 1476.0, 2127400)
(1306, '2007/11/20', 1441.0, 1491.0, 1438.0, 1488.0, 3261700)
(1306, '2007/11/21', 1481.0, 1487.0, 1452.0, 1456.0, 2698900)
(1306, '2007/11/22', 1436.0, 1466.0, 1435.0, 1457.0, 4162200)
(1306, '2007/11/26', 1467.0, 1500.0, 1459.0, 1485.0, 3002200)
(1306, '2007/11/27', 1454.0, 1505.0, 1450.0, 1500.0, 2319800)
(1306, '2007/11/28', 1507.0, 1509.0, 1488.0, 1494.0, 1869600)
(1306, '2007/11/29', 1535.0, 1539.0, 1529.0, 1532.0, 3924400)
(1306, '2007/11/30', 1533.0, 1557.0, 1531.0, 1549.0, 3027100)
(1306, '2007/12/03', 1561.0, 1570.0, 1544.0, 1554.0, 2181700)
(1306, '2007/12/04', 1554.0, 1556.0, 1531.0, 1534.0, 1771200)
(1306, '2007/12/05', 1524.0, 1548.0, 1520.0, 1547.0, 984100)
(1306, '2007/12/06', 1572.0, 1573.0, 1555.0, 1573.0, 2236300)
(1306, '2007/12/07', 1589.0, 1598.0, 1578.0, 1580.0, 2165200)
(1306, '2007/12/10', 1583.0, 1588.0, 1569.0, 1578.0, 5720400)
(1306, '2

(1306, '2015/02/16', 1490.0, 1498.0, 1487.0, 1493.0, 1899780)
(1306, '2015/02/17', 1490.0, 1497.0, 1486.0, 1495.0, 2164080)
(1306, '2015/02/18', 1509.0, 1519.0, 1507.0, 1517.0, 3168370)
(1306, '2015/02/19', 1520.0, 1531.0, 1520.0, 1529.0, 4048450)
(1306, '2015/02/20', 1534.0, 1536.0, 1528.0, 1534.0, 2300000)
(1306, '2015/02/23', 1546.0, 1548.0, 1529.0, 1537.0, 3909030)
(1306, '2015/02/24', 1538.0, 1544.0, 1532.0, 1543.0, 3115490)
(1306, '2015/02/25', 1549.0, 1550.0, 1540.0, 1544.0, 7431260)
(1306, '2015/02/26', 1541.0, 1558.0, 1541.0, 1558.0, 4110800)
(1306, '2015/02/27', 1561.0, 1565.0, 1553.0, 1560.0, 4087360)
(1306, '2015/03/02', 1562.0, 1569.0, 1559.0, 1560.0, 2728540)
(1306, '2015/03/03', 1569.0, 1569.0, 1553.0, 1564.0, 2586430)
(1306, '2015/03/04', 1557.0, 1557.0, 1538.0, 1554.0, 3150660)
(1306, '2015/03/05', 1551.0, 1561.0, 1550.0, 1560.0, 3398180)
(1306, '2015/03/06', 1561.0, 1577.0, 1561.0, 1577.0, 4238980)
(1306, '2015/03/09', 1570.0, 1572.0, 1560.0, 1566.0, 2507570)
(1306, '

(1308, '2003/03/19', 787.0, 792.0, 776.0, 792.0, 1362000)
(1308, '2003/03/20', 800.0, 814.0, 798.0, 809.0, 2152000)
(1308, '2003/03/24', 819.0, 838.0, 819.0, 835.0, 2980000)
(1308, '2003/03/25', 825.0, 829.0, 814.0, 814.0, 1781000)
(1308, '2003/03/26', 828.0, 832.0, 822.0, 832.0, 2176000)
(1308, '2003/03/27', 826.0, 835.0, 826.0, 832.0, 856000)
(1308, '2003/03/28', 830.0, 833.0, 825.0, 826.0, 826000)
(1308, '2003/03/31', 820.0, 821.0, 795.0, 795.0, 1353000)
(1308, '2003/04/01', 789.0, 803.0, 785.0, 800.0, 2692000)
(1308, '2003/04/02', 803.0, 805.0, 792.0, 804.0, 1143000)
(1308, '2003/04/03', 814.0, 817.0, 800.0, 802.0, 1858000)
(1308, '2003/04/04', 802.0, 808.0, 797.0, 807.0, 1794000)
(1308, '2003/04/07', 811.0, 820.0, 800.0, 820.0, 3122000)
(1308, '2003/04/08', 815.0, 817.0, 807.0, 809.0, 1935000)
(1308, '2003/04/09', 809.0, 815.0, 805.0, 806.0, 2559000)
(1308, '2003/04/10', 805.0, 807.0, 798.0, 801.0, 2915000)
(1308, '2003/04/11', 799.0, 801.0, 787.0, 791.0, 3812000)
(1308, '2003/04/

(1308, '2008/06/11', 1419.0, 1420.0, 1404.0, 1419.0, 430000)
(1308, '2008/06/12', 1397.0, 1397.0, 1386.0, 1390.0, 941000)
(1308, '2008/06/13', 1407.0, 1407.0, 1381.0, 1401.0, 508000)
(1308, '2008/06/16', 1416.0, 1430.0, 1410.0, 1430.0, 119000)
(1308, '2008/06/17', 1437.0, 1437.0, 1425.0, 1430.0, 185000)
(1308, '2008/06/18', 1427.0, 1440.0, 1427.0, 1438.0, 323000)
(1308, '2008/06/19', 1423.0, 1424.0, 1397.0, 1401.0, 798000)
(1308, '2008/06/20', 1406.0, 1409.0, 1383.0, 1383.0, 260000)
(1308, '2008/06/23', 1365.0, 1378.0, 1356.0, 1376.0, 360000)
(1308, '2008/06/24', 1372.0, 1380.0, 1368.0, 1377.0, 483000)
(1308, '2008/06/25', 1374.0, 1376.0, 1354.0, 1375.0, 175000)
(1308, '2008/06/26', 1378.0, 1384.0, 1370.0, 1372.0, 67000)
(1308, '2008/06/27', 1336.0, 1350.0, 1336.0, 1348.0, 279000)
(1308, '2008/06/30', 1354.0, 1360.0, 1347.0, 1347.0, 998000)
(1308, '2008/07/01', 1352.0, 1361.0, 1347.0, 1349.0, 810000)
(1308, '2008/07/02', 1352.0, 1352.0, 1326.0, 1330.0, 311000)
(1308, '2008/07/03', 1299

(1308, '2014/09/24', 1325.0, 1333.0, 1325.0, 1331.0, 634300)
(1308, '2014/09/25', 1343.0, 1351.0, 1342.0, 1349.0, 718900)
(1308, '2014/09/26', 1338.0, 1347.0, 1336.0, 1346.0, 439500)
(1308, '2014/09/29', 1353.0, 1354.0, 1349.0, 1351.0, 424300)
(1308, '2014/09/30', 1347.0, 1347.0, 1329.0, 1338.0, 1677100)
(1308, '2014/10/01', 1340.0, 1344.0, 1331.0, 1331.0, 509200)
(1308, '2014/10/02', 1316.0, 1317.0, 1292.0, 1294.0, 2226300)
(1308, '2014/10/03', 1288.0, 1298.0, 1287.0, 1296.0, 1658000)
(1308, '2014/10/06', 1313.0, 1317.0, 1306.0, 1311.0, 535500)
(1308, '2014/10/07', 1311.0, 1317.0, 1303.0, 1306.0, 401800)
(1308, '2014/10/08', 1283.0, 1292.0, 1282.0, 1289.0, 1074400)
(1308, '2014/10/09', 1294.0, 1296.0, 1273.0, 1275.0, 824600)
(1308, '2014/10/10', 1250.0, 1259.0, 1250.0, 1256.0, 1391800)
(1308, '2014/10/14', 1226.0, 1238.0, 1225.0, 1229.0, 919500)
(1308, '2014/10/15', 1231.0, 1239.0, 1225.0, 1237.0, 348800)
(1308, '2014/10/16', 1211.0, 1214.0, 1205.0, 1208.0, 2028800)
(1308, '2014/10/17

## 複雑なSQL

In [50]:
t = text("SELECT date FROM history")
rows = engine.execute(t)
for row in rows:
    print(row)

2018-01-22 02:17:23,393 INFO sqlalchemy.engine.base.Engine SELECT date FROM history
2018-01-22 02:17:23,395 INFO sqlalchemy.engine.base.Engine ()
('2000/01/04',)
('2000/01/05',)
('2000/01/06',)
('2000/01/07',)
('2000/01/11',)
('2000/01/12',)
('2000/01/13',)
('2000/01/14',)
('2000/01/17',)
('2000/01/18',)
('2000/01/19',)
('2000/01/20',)
('2000/01/21',)
('2000/01/24',)
('2000/01/25',)
('2000/01/26',)
('2000/01/27',)
('2000/01/28',)
('2000/01/31',)
('2000/02/01',)
('2000/02/02',)
('2000/02/03',)
('2000/02/04',)
('2000/02/07',)
('2000/02/08',)
('2000/02/09',)
('2000/02/10',)
('2000/02/14',)
('2000/02/15',)
('2000/02/16',)
('2000/02/17',)
('2000/02/18',)
('2000/02/21',)
('2000/02/22',)
('2000/02/23',)
('2000/02/24',)
('2000/02/25',)
('2000/02/28',)
('2000/02/29',)
('2000/03/01',)
('2000/03/02',)
('2000/03/03',)
('2000/03/06',)
('2000/03/07',)
('2000/03/08',)
('2000/03/09',)
('2000/03/10',)
('2000/03/13',)
('2000/03/14',)
('2000/03/15',)
('2000/03/16',)
('2000/03/17',)
('2000/03/21',)
('2000

('2004/07/20',)
('2004/07/21',)
('2004/07/22',)
('2004/07/23',)
('2004/07/26',)
('2004/07/27',)
('2004/07/28',)
('2004/07/29',)
('2004/07/30',)
('2004/08/02',)
('2004/08/03',)
('2004/08/04',)
('2004/08/05',)
('2004/08/06',)
('2004/08/09',)
('2004/08/10',)
('2004/08/11',)
('2004/08/12',)
('2004/08/13',)
('2004/08/16',)
('2004/08/17',)
('2004/08/18',)
('2004/08/19',)
('2004/08/20',)
('2004/08/23',)
('2004/08/24',)
('2004/08/25',)
('2004/08/26',)
('2004/08/27',)
('2004/08/30',)
('2004/08/31',)
('2004/09/01',)
('2004/09/02',)
('2004/09/03',)
('2004/09/06',)
('2004/09/07',)
('2004/09/08',)
('2004/09/09',)
('2004/09/10',)
('2004/09/13',)
('2004/09/14',)
('2004/09/15',)
('2004/09/16',)
('2004/09/17',)
('2004/09/21',)
('2004/09/22',)
('2004/09/24',)
('2004/09/27',)
('2004/09/28',)
('2004/09/29',)
('2004/09/30',)
('2004/10/01',)
('2004/10/04',)
('2004/10/05',)
('2004/10/06',)
('2004/10/07',)
('2004/10/08',)
('2004/10/12',)
('2004/10/13',)
('2004/10/14',)
('2004/10/15',)
('2004/10/18',)
('2004/1

('2016/01/06',)
('2016/01/07',)
('2016/01/08',)
('2016/01/12',)
('2016/01/13',)
('2016/01/14',)
('2016/01/15',)
('2016/01/18',)
('2016/01/19',)
('2016/01/20',)
('2016/01/21',)
('2016/01/22',)
('2016/01/25',)
('2016/01/26',)
('2016/01/27',)
('2016/01/28',)
('2016/01/29',)
('2016/02/01',)
('2016/02/02',)
('2016/02/03',)
('2016/02/04',)
('2016/02/05',)
('2016/02/08',)
('2016/02/09',)
('2016/02/10',)
('2016/02/12',)
('2016/02/15',)
('2016/02/16',)
('2016/02/17',)
('2016/02/18',)
('2016/02/19',)
('2016/02/22',)
('2016/02/23',)
('2016/02/24',)
('2016/02/25',)
('2016/02/26',)
('2016/02/29',)
('2016/03/01',)
('2016/03/02',)
('2016/03/03',)
('2016/03/04',)
('2016/03/07',)
('2016/03/08',)
('2016/03/09',)
('2016/03/10',)
('2016/03/11',)
('2016/03/14',)
('2016/03/15',)
('2016/03/16',)
('2016/03/17',)
('2016/03/18',)
('2016/03/22',)
('2016/03/23',)
('2016/03/24',)
('2016/03/25',)
('2016/03/28',)
('2016/03/29',)
('2016/03/30',)
('2016/03/31',)
('2016/04/01',)
('2016/04/04',)
('2016/04/05',)
('2016/0

('2008/02/15',)
('2008/02/18',)
('2008/02/19',)
('2008/02/20',)
('2008/02/21',)
('2008/02/22',)
('2008/02/25',)
('2008/02/26',)
('2008/02/27',)
('2008/02/28',)
('2008/02/29',)
('2008/03/03',)
('2008/03/04',)
('2008/03/05',)
('2008/03/06',)
('2008/03/07',)
('2008/03/10',)
('2008/03/11',)
('2008/03/12',)
('2008/03/13',)
('2008/03/14',)
('2008/03/17',)
('2008/03/18',)
('2008/03/19',)
('2008/03/21',)
('2008/03/24',)
('2008/03/25',)
('2008/03/26',)
('2008/03/27',)
('2008/03/28',)
('2008/03/31',)
('2008/04/01',)
('2008/04/02',)
('2008/04/03',)
('2008/04/04',)
('2008/04/07',)
('2008/04/08',)
('2008/04/09',)
('2008/04/10',)
('2008/04/11',)
('2008/04/14',)
('2008/04/15',)
('2008/04/16',)
('2008/04/17',)
('2008/04/18',)
('2008/04/21',)
('2008/04/22',)
('2008/04/23',)
('2008/04/24',)
('2008/04/25',)
('2008/04/28',)
('2008/04/30',)
('2008/05/01',)
('2008/05/02',)
('2008/05/07',)
('2008/05/08',)
('2008/05/09',)
('2008/05/12',)
('2008/05/13',)
('2008/05/14',)
('2008/05/15',)
('2008/05/16',)
('2008/0

('2017/01/13',)
('2017/01/16',)
('2017/01/17',)
('2017/01/18',)
('2017/01/19',)
('2017/01/20',)
('2017/01/23',)
('2017/01/24',)
('2017/01/25',)
('2017/01/26',)
('2017/01/27',)
('2017/01/30',)
('2017/01/31',)
('2017/02/01',)
('2017/02/02',)
('2017/02/03',)
('2017/02/06',)
('2017/02/07',)
('2017/02/08',)
('2017/02/09',)
('2017/02/10',)
('2017/02/13',)
('2017/02/14',)
('2017/02/15',)
('2017/02/16',)
('2017/02/17',)
('2017/02/20',)
('2017/02/21',)
('2017/02/22',)
('2017/02/23',)
('2017/02/24',)
('2017/02/27',)
('2017/02/28',)
('2017/03/01',)
('2017/03/02',)
('2017/03/03',)
('2017/03/06',)
('2017/03/07',)
('2017/03/08',)
('2017/03/09',)
('2017/03/10',)
('2017/03/13',)
('2017/03/14',)
('2017/03/15',)
('2017/03/16',)
('2017/03/17',)
('2017/03/21',)
('2017/03/22',)
('2017/03/23',)
('2017/03/24',)
('2017/03/27',)
('2017/03/28',)
('2017/03/29',)
('2017/03/30',)
('2017/03/31',)
('2017/04/03',)
('2017/04/04',)
('2017/04/05',)
('2017/04/06',)
('2017/04/07',)
('2017/04/10',)
('2017/04/11',)
('2017/0

('2007/08/07',)
('2007/08/08',)
('2007/08/09',)
('2007/08/10',)
('2007/08/13',)
('2007/08/14',)
('2007/08/15',)
('2007/08/16',)
('2007/08/17',)
('2007/08/20',)
('2007/08/21',)
('2007/08/22',)
('2007/08/23',)
('2007/08/24',)
('2007/08/27',)
('2007/08/28',)
('2007/08/29',)
('2007/08/30',)
('2007/08/31',)
('2007/09/03',)
('2007/09/04',)
('2007/09/05',)
('2007/09/06',)
('2007/09/07',)
('2007/09/10',)
('2007/09/11',)
('2007/09/12',)
('2007/09/13',)
('2007/09/14',)
('2007/09/18',)
('2007/09/19',)
('2007/09/20',)
('2007/09/21',)
('2007/09/25',)
('2007/09/26',)
('2007/09/27',)
('2007/09/28',)
('2007/10/01',)
('2007/10/02',)
('2007/10/03',)
('2007/10/04',)
('2007/10/05',)
('2007/10/09',)
('2007/10/10',)
('2007/10/11',)
('2007/10/12',)
('2007/10/15',)
('2007/10/16',)
('2007/10/17',)
('2007/10/18',)
('2007/10/19',)
('2007/10/22',)
('2007/10/23',)
('2007/10/24',)
('2007/10/25',)
('2007/10/26',)
('2007/10/29',)
('2007/10/30',)
('2007/10/31',)
('2007/11/01',)
('2007/11/02',)
('2007/11/05',)
('2007/1

('2002/03/29',)
('2002/04/01',)
('2002/04/02',)
('2002/04/03',)
('2002/04/04',)
('2002/04/05',)
('2002/04/08',)
('2002/04/09',)
('2002/04/10',)
('2002/04/11',)
('2002/04/12',)
('2002/04/15',)
('2002/04/16',)
('2002/04/17',)
('2002/04/18',)
('2002/04/19',)
('2002/04/22',)
('2002/04/23',)
('2002/04/24',)
('2002/04/25',)
('2002/04/26',)
('2002/04/30',)
('2002/05/01',)
('2002/05/02',)
('2002/05/07',)
('2002/05/08',)
('2002/05/09',)
('2002/05/10',)
('2002/05/13',)
('2002/05/14',)
('2002/05/15',)
('2002/05/16',)
('2002/05/17',)
('2002/05/20',)
('2002/05/21',)
('2002/05/22',)
('2002/05/23',)
('2002/05/24',)
('2002/05/27',)
('2002/05/28',)
('2002/05/29',)
('2002/05/30',)
('2002/05/31',)
('2002/06/03',)
('2002/06/04',)
('2002/06/05',)
('2002/06/06',)
('2002/06/07',)
('2002/06/10',)
('2002/06/11',)
('2002/06/12',)
('2002/06/13',)
('2002/06/14',)
('2002/06/17',)
('2002/06/18',)
('2002/06/19',)
('2002/06/20',)
('2002/06/21',)
('2002/06/24',)
('2002/06/25',)
('2002/06/26',)
('2002/06/27',)
('2002/0

('2010/10/28',)
('2010/10/29',)
('2010/11/01',)
('2010/11/02',)
('2010/11/04',)
('2010/11/05',)
('2010/11/08',)
('2010/11/09',)
('2010/11/10',)
('2010/11/11',)
('2010/11/12',)
('2010/11/15',)
('2010/11/16',)
('2010/11/17',)
('2010/11/18',)
('2010/11/19',)
('2010/11/22',)
('2010/11/24',)
('2010/11/25',)
('2010/11/26',)
('2010/11/29',)
('2010/11/30',)
('2010/12/01',)
('2010/12/02',)
('2010/12/03',)
('2010/12/06',)
('2010/12/07',)
('2010/12/08',)
('2010/12/09',)
('2010/12/10',)
('2010/12/13',)
('2010/12/14',)
('2010/12/15',)
('2010/12/16',)
('2010/12/17',)
('2010/12/20',)
('2010/12/21',)
('2010/12/22',)
('2010/12/24',)
('2010/12/27',)
('2010/12/28',)
('2010/12/29',)
('2010/12/30',)
('2011/01/04',)
('2011/01/05',)
('2011/01/06',)
('2011/01/07',)
('2011/01/11',)
('2011/01/12',)
('2011/01/13',)
('2011/01/14',)
('2011/01/17',)
('2011/01/18',)
('2011/01/19',)
('2011/01/20',)
('2011/01/21',)
('2011/01/24',)
('2011/01/25',)
('2011/01/26',)
('2011/01/27',)
('2011/01/28',)
('2011/01/31',)
('2011/0

In [54]:
t = text("SELECT * FROM history WHERE date BETWEEN '2004/01/01' AND '2004/01/08'")
rows = engine.execute(t)
for row in rows:
    print(row)

2018-01-22 02:19:52,152 INFO sqlalchemy.engine.base.Engine SELECT * FROM history WHERE date BETWEEN '2004/01/01' AND '2004/01/08'
2018-01-22 02:19:52,153 INFO sqlalchemy.engine.base.Engine ()
(1301, '2004/01/05', 1710.0, 1720.0, 1690.0, 1690.0, 384000)
(1301, '2004/01/06', 1700.0, 1700.0, 1660.0, 1670.0, 475000)
(1301, '2004/01/07', 1670.0, 1770.0, 1670.0, 1760.0, 2264000)
(1301, '2004/01/08', 1760.0, 1780.0, 1730.0, 1770.0, 1016000)
(1305, '2004/01/05', 1070.0, 1074.0, 1068.0, 1070.0, 90100)
(1305, '2004/01/06', 1080.0, 1082.0, 1066.0, 1066.0, 116500)
(1305, '2004/01/07', 1066.0, 1067.0, 1059.0, 1061.0, 61800)
(1305, '2004/01/08', 1062.0, 1076.0, 1062.0, 1070.0, 15200)
(1306, '2004/01/05', 1073.0, 1074.0, 1067.0, 1071.0, 917100)
(1306, '2004/01/06', 1086.0, 1086.0, 1065.0, 1066.0, 1357200)
(1306, '2004/01/07', 1068.0, 1068.0, 1057.0, 1060.0, 623600)
(1306, '2004/01/08', 1064.0, 1074.0, 1062.0, 1067.0, 1272700)
(1308, '2004/01/05', 1069.0, 1070.0, 1063.0, 1067.0, 788000)
(1308, '2004/0

## 動的SQL発行

In [59]:
start_date = "'2004/01/01'"
end_date = "'2004/01/08'"

上と同じSQLを発行するためには日付を二重に囲う必要がある。
また、 BETWWEN 日付 AND 日付　とするために空白も必要である。 

In [60]:
t = text("SELECT * FROM history WHERE date BETWEEN " + start_date + " AND " + end_date )
rows = engine.execute(t)
for row in rows:
    print(row)

2018-01-22 04:12:24,935 INFO sqlalchemy.engine.base.Engine SELECT * FROM history WHERE date BETWEEN '2004/01/01' AND '2004/01/08'
2018-01-22 04:12:24,936 INFO sqlalchemy.engine.base.Engine ()
(1301, '2004/01/05', 1710.0, 1720.0, 1690.0, 1690.0, 384000)
(1301, '2004/01/06', 1700.0, 1700.0, 1660.0, 1670.0, 475000)
(1301, '2004/01/07', 1670.0, 1770.0, 1670.0, 1760.0, 2264000)
(1301, '2004/01/08', 1760.0, 1780.0, 1730.0, 1770.0, 1016000)
(1305, '2004/01/05', 1070.0, 1074.0, 1068.0, 1070.0, 90100)
(1305, '2004/01/06', 1080.0, 1082.0, 1066.0, 1066.0, 116500)
(1305, '2004/01/07', 1066.0, 1067.0, 1059.0, 1061.0, 61800)
(1305, '2004/01/08', 1062.0, 1076.0, 1062.0, 1070.0, 15200)
(1306, '2004/01/05', 1073.0, 1074.0, 1067.0, 1071.0, 917100)
(1306, '2004/01/06', 1086.0, 1086.0, 1065.0, 1066.0, 1357200)
(1306, '2004/01/07', 1068.0, 1068.0, 1057.0, 1060.0, 623600)
(1306, '2004/01/08', 1064.0, 1074.0, 1062.0, 1067.0, 1272700)
(1308, '2004/01/05', 1069.0, 1070.0, 1063.0, 1067.0, 788000)
(1308, '2004/0

## より簡潔な動的SQL(予定)
### Datetimeとかで日付とかもっと分かりやすくしたい